In [64]:
import numpy as np
from ENDFtk.tree import Tape
import h5py, os
import numpy as np
import importlib
import CovarianceResonance
importlib.reload(CovarianceResonance)
from CovarianceResonance import ResonanceCovariance as RC

# Read the ENDF file
# tapeMBCnURR = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-026_Fe_055.endf')    # OK
tapeRMCnURR = Tape.from_file('/home/sole-pie01/work/ndlib/tendl17-neutron/n-U235-s20.tendl.txt')# OK
# tapeRM = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-094_Pu_239.endf')         # OK
# tapeRMLC = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-029_Cu_063.endf')       # OK

tape = tapeRMCnURR
mf2 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
mf32 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

# Get the resonance ranges from MF2 and MF32
mf2_resonance_ranges = mf2.isotopes[0].resonance_ranges.to_list()
mf32_resonance_ranges = mf32.isotopes[0].resonance_ranges.to_list()

NER = 1
covariance_obj = RC.from_resonance_range(mf32_resonance_ranges[NER], mf2_resonance_ranges, NER)
# Access the covariance matrix and parameters
covariance_matrix = covariance_obj.covariance_matrix
parameters = covariance_obj.parameters
# Use the data as needed
print(f"\nCovariance matrix for NER={NER}:\n{covariance_matrix[:4, :4]}")
covariance_obj.print_parameters()



Covariance matrix for NER=1:
[[1.02320306e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.82813175e-13 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.92779388e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.30018653e-13]]
Parameters for NER=1:
Index: 0, L: 0, J: 3.0, Type: D, Mean: 1.011535
Index: 1, L: 0, J: 3.0, Type: GNO, Mean: 8.847673e-06
Index: 2, L: 0, J: 4.0, Type: D, Mean: 0.8323337
Index: 3, L: 0, J: 4.0, Type: GNO, Mean: 7.280238e-06
Index: 4, L: 1, J: 2.0, Type: D, Mean: 1.357573
Index: 5, L: 1, J: 2.0, Type: GNO, Mean: 2.526229e-05
Index: 6, L: 1, J: 3.0, Type: D, Mean: 1.011535
Index: 7, L: 1, J: 3.0, Type: GNO, Mean: 1.882307e-05
Index: 8, L: 1, J: 4.0, Type: D, Mean: 0.8323337
Index: 9, L: 1, J: 4.0, Type: GNO, Mean: 1.548841e-05
Index: 10, L: 1, J: 5.0, Type: D, Mean: 0.7306738
Index: 11, L: 1, J: 5.0, Type: GNO, Mean: 1.359668e-05


In [ ]:
from ENDFtk.MF2.MT151 import ParticlePairs
from ENDFtk.MF2.MT151 import ResonanceChannels
from ENDFtk.MF2.MT151 import ResonanceParameters
from ENDFtk.MF2.MT151 import SpinGroup
from ENDFtk.MF2.MT151 import RMatrixLimited
from ENDFtk.MF2.MT151 import ResonanceRange
from ENDFtk.MF2.MT151 import Isotope
from ENDFtk.MF2.MT151 import Section


MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()

isotopeIndex = 0 # Always zero
rangeIndex = 0 # User defined
RANGE = MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex]
pairs = ParticlePairs( RANGE.parameters.particle_pairs )

groups = []
for sgroup in RANGE.parameters.spin_groups.to_list():
    groups.append( SpinGroup( ResonanceChannels( sgroup.channels ),
                              ResonanceParameters( sgroup.parameters.ER, sgroup.parameters.GAM ) ))
    

parameters = RMatrixLimited( ifg = RANGE.parameters.IFG, 
                            krl = RANGE.parameters.KRL, 
                            krm = RANGE.parameters.KRM,
                            pairs = pairs, groups = groups )

isotopes = [ Isotope( zai = MATMF2MT151.isotopes[isotopeIndex].ZAI, 
                      abn = MATMF2MT151.isotopes[isotopeIndex].ABN, 
                      lfw = MATMF2MT151.isotopes[isotopeIndex].LFW,
                      ranges = [ ResonanceRange( el = RANGE.EL, 
                                                 eh = RANGE.EH,
                                                 naps = RANGE.NAPS, 
                                                 parameters = parameters ) ] ) ]
dummySection = Section( zaid = MATMF2MT151.ZA, awr = MATMF2MT151.AWR, isotopes = isotopes )

tape.MAT(tape.material_numbers[0]).MF(2).insert_or_replace( dummySection )
tape.to_file("toto.endf")

In [ ]:
for spingroup in mf2_resonance_ranges[0].parameters.spin_groups.to_list():
    print("---------------", spingroup.AJ, spingroup.PJ) # Each spin group has a Spin AJ and a Parity PJ 
    # There will be NRS resonances and 
    for resonance in range(spingroup.parameters.NRS):
        print(spingroup.parameters.ER[resonance], [spingroup.parameters.GAM[resonance][channel] for channel in range(spingroup.NCH)])

# Reading MF32, LRF=7, LCOMP=2

# mf32_resonance_ranges[0].parameters.NJS # number of spin groups
mf32_resonance_ranges[0].parameters.ISR 
# If ISR is True there are uncertainties on scattering radii AP called DAP
# Note that every spingroup has NCH channels, each channels have an AP and can have a DAP. Create flexible object to store 

for spingroup in mf32_resonance_ranges[0].parameters.uncertainties.spin_groups.to_list():
    print("---------------", spingroup.AJ, spingroup.PJ) # Each spin group has a Spin AJ and a Parity PJ 
    for resonance in range(spingroup.NRSA):
        # Then each parameter can be positionned with it corresponding resonance number in MF2 and channel number
        print(spingroup.parameters.ER[resonance], [spingroup.parameters.GAM[resonance][channel] for channel in range(spingroup.NCH)])
        # As for other LCOMP=2, we will have to retrieve the standard deviation vector made of successive resonance parameters (DER std.dev on energy and as much std.dev on widths DGAM as channel is this spingroup) 
        print(spingroup.parameters.DER[resonance], [spingroup.parameters.DGAM[resonance][channel] for channel in range(spingroup.NCH)])
        
mf32_resonance_ranges[0].parameters.correlation_matrix.I
mf32_resonance_ranges[0].parameters.correlation_matrix.J
mf32_resonance_ranges[0].parameters.correlation_matrix.correlations

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [ ]:
# For each resonance range in MF32, create the covariance object
# for NER, resonance_range in enumerate(mf32_resonance_ranges):
#     LRU = resonance_range.LRU
#     LRF = resonance_range.LRF
#     if LRU == 1 and LRF == 3:
#         covariance_obj = ResonanceCovariance.from_resonance_range(resonance_range, mf2_resonance_ranges, NER)
#     elif LRU == 1 and LRF == 2:
#         covariance_obj = ResonanceCovariance.from_resonance_range(resonance_range, mf2_resonance_ranges, NER)
#     elif LRU == 2 and LRF == 0:
#         covariance_obj = ResonanceCovariance.from_resonance_range(resonance_range, None, NER)
#     else:
#         continue  # Skip unsupported formats

#     # Access the covariance matrix and parameters
#     covariance_matrix = covariance_obj.covariance_matrix
#     parameters = covariance_obj.parameters

#     # Use the data as needed
#     print(f"\nCovariance matrix for NER={NER}:\n{covariance_matrix[:5, :5]}")
#     covariance_obj.print_parameters()

# For each resonance range in MF32, create the covariance object
# for NER, resonance_range in enumerate(mf32_resonance_ranges):
#     if resonance_range.LRU == 1 and resonance_range.LRF == 2:
#         covariance_obj = ResonanceCovariance.from_resonance_range(resonance_range, mf2_resonance_ranges, NER)
#     elif resonance_range.LRU == 2 and resonance_range.LRF == 1:
#         covariance_obj = ResonanceCovariance.from_resonance_range(resonance_range, None, NER)
#     else:
#         continue  # Skip unsupported formats

#     # Access the covariance matrix and parameters
#     covariance_matrix = covariance_obj.covariance_matrix
#     parameters = covariance_obj.parameters

#     # Use the data as needed
#     print(f"\nCovariance matrix for NER={NER}:\n{covariance_matrix[:6, :6]}")
#     covariance_obj.print_parameters()